# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f19c8395310>
├── 'a' --> tensor([[-0.3284, -0.3498, -0.5028],
│                   [-0.1772, -0.1437,  1.2174]])
└── 'x' --> <FastTreeValue 0x7f19c8395130>
    └── 'c' --> tensor([[ 1.2162,  0.1717, -0.4382,  0.6088],
                        [-0.5225, -0.0738, -1.2179,  1.6785],
                        [-0.5423,  0.0741,  0.7471, -0.5535]])

In [4]:
t.a

tensor([[-0.3284, -0.3498, -0.5028],
        [-0.1772, -0.1437,  1.2174]])

In [5]:
%timeit t.a

59.1 ns ± 0.0574 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f19c8395310>
├── 'a' --> tensor([[ 1.2886, -1.3147,  0.3933],
│                   [ 0.7399,  0.7805,  1.6314]])
└── 'x' --> <FastTreeValue 0x7f19c8395130>
    └── 'c' --> tensor([[ 1.2162,  0.1717, -0.4382,  0.6088],
                        [-0.5225, -0.0738, -1.2179,  1.6785],
                        [-0.5423,  0.0741,  0.7471, -0.5535]])

In [7]:
%timeit t.a = new_value

65.2 ns ± 0.0282 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3284, -0.3498, -0.5028],
               [-0.1772, -0.1437,  1.2174]]),
    x: Batch(
           c: tensor([[ 1.2162,  0.1717, -0.4382,  0.6088],
                      [-0.5225, -0.0738, -1.2179,  1.6785],
                      [-0.5423,  0.0741,  0.7471, -0.5535]]),
       ),
)

In [10]:
b.a

tensor([[-0.3284, -0.3498, -0.5028],
        [-0.1772, -0.1437,  1.2174]])

In [11]:
%timeit b.a

54.3 ns ± 0.0371 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3814,  0.2009, -0.6955],
               [ 0.0632,  1.0818, -1.2763]]),
    x: Batch(
           c: tensor([[ 1.2162,  0.1717, -0.4382,  0.6088],
                      [-0.5225, -0.0738, -1.2179,  1.6785],
                      [-0.5423,  0.0741,  0.7471, -0.5535]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.292 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

831 ns ± 0.849 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 48.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 833 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 396 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f19c8395820>
├── 'a' --> tensor([[[-0.3284, -0.3498, -0.5028],
│                    [-0.1772, -0.1437,  1.2174]],
│           
│                   [[-0.3284, -0.3498, -0.5028],
│                    [-0.1772, -0.1437,  1.2174]],
│           
│                   [[-0.3284, -0.3498, -0.5028],
│                    [-0.1772, -0.1437,  1.2174]],
│           
│                   [[-0.3284, -0.3498, -0.5028],
│                    [-0.1772, -0.1437,  1.2174]],
│           
│                   [[-0.3284, -0.3498, -0.5028],
│                    [-0.1772, -0.1437,  1.2174]],
│           
│                   [[-0.3284, -0.3498, -0.5028],
│                    [-0.1772, -0.1437,  1.2174]],
│           
│                   [[-0.3284, -0.3498, -0.5028],
│                    [-0.1772, -0.1437,  1.2174]],
│           
│                   [[-0.3284, -0.3498, -0.5028],
│                    [-0.1772, -0.1437,  1.2174]]])
└── 'x' --> <FastTreeValue 0x7f19c8395220>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 79.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f19c839b970>
├── 'a' --> tensor([[-0.3284, -0.3498, -0.5028],
│                   [-0.1772, -0.1437,  1.2174],
│                   [-0.3284, -0.3498, -0.5028],
│                   [-0.1772, -0.1437,  1.2174],
│                   [-0.3284, -0.3498, -0.5028],
│                   [-0.1772, -0.1437,  1.2174],
│                   [-0.3284, -0.3498, -0.5028],
│                   [-0.1772, -0.1437,  1.2174],
│                   [-0.3284, -0.3498, -0.5028],
│                   [-0.1772, -0.1437,  1.2174],
│                   [-0.3284, -0.3498, -0.5028],
│                   [-0.1772, -0.1437,  1.2174],
│                   [-0.3284, -0.3498, -0.5028],
│                   [-0.1772, -0.1437,  1.2174],
│                   [-0.3284, -0.3498, -0.5028],
│                   [-0.1772, -0.1437,  1.2174]])
└── 'x' --> <FastTreeValue 0x7f19c839b850>
    └── 'c' --> tensor([[ 1.2162,  0.1717, -0.4382,  0.6088],
                        [-0.5225, -0.0738, -1.2179,  1.6785],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 62.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3284, -0.3498, -0.5028],
                [-0.1772, -0.1437,  1.2174]],
       
               [[-0.3284, -0.3498, -0.5028],
                [-0.1772, -0.1437,  1.2174]],
       
               [[-0.3284, -0.3498, -0.5028],
                [-0.1772, -0.1437,  1.2174]],
       
               [[-0.3284, -0.3498, -0.5028],
                [-0.1772, -0.1437,  1.2174]],
       
               [[-0.3284, -0.3498, -0.5028],
                [-0.1772, -0.1437,  1.2174]],
       
               [[-0.3284, -0.3498, -0.5028],
                [-0.1772, -0.1437,  1.2174]],
       
               [[-0.3284, -0.3498, -0.5028],
                [-0.1772, -0.1437,  1.2174]],
       
               [[-0.3284, -0.3498, -0.5028],
                [-0.1772, -0.1437,  1.2174]]]),
    x: Batch(
           c: tensor([[[ 1.2162,  0.1717, -0.4382,  0.6088],
                       [-0.5225, -0.0738, -1.2179,  1.6785],
                       [-0.5423,  0.0741,  0.7471, -0.5535]],
         

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 162 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3284, -0.3498, -0.5028],
               [-0.1772, -0.1437,  1.2174],
               [-0.3284, -0.3498, -0.5028],
               [-0.1772, -0.1437,  1.2174],
               [-0.3284, -0.3498, -0.5028],
               [-0.1772, -0.1437,  1.2174],
               [-0.3284, -0.3498, -0.5028],
               [-0.1772, -0.1437,  1.2174],
               [-0.3284, -0.3498, -0.5028],
               [-0.1772, -0.1437,  1.2174],
               [-0.3284, -0.3498, -0.5028],
               [-0.1772, -0.1437,  1.2174],
               [-0.3284, -0.3498, -0.5028],
               [-0.1772, -0.1437,  1.2174],
               [-0.3284, -0.3498, -0.5028],
               [-0.1772, -0.1437,  1.2174]]),
    x: Batch(
           c: tensor([[ 1.2162,  0.1717, -0.4382,  0.6088],
                      [-0.5225, -0.0738, -1.2179,  1.6785],
                      [-0.5423,  0.0741,  0.7471, -0.5535],
                      [ 1.2162,  0.1717, -0.4382,  0.6088],
                      [-0.5225, -

In [28]:
%timeit Batch.cat(batches)

141 µs ± 439 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

315 µs ± 2.62 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
